# Libraries

In [1]:
# importing the necessary libraries
import scipy as sp
import numpy as np
import pandas as pd
import sklearn as sk
from collections import defaultdict

try:
    import py_entitymatching as em
except:
    !pip install py_entitymatching

try:
    import deepmatcher as dm
except:
    !pip uninstall preprocessing -y
    !pip uninstall fastai -y 
    !pip uninstall allennlp -y
    !pip install deepmatcher

if pd.__version__ != "1.2.4":
 !pip install pandas==1.2.4
 import pandas as pd
 print(f"Verify pandas=={pd.__version__}")
else:
 print(f"Verify pandas=={pd.__version__}")

import sys
import tensorflow as tf
from tensorflow.python.client import device_lib
import os
from imblearn.under_sampling import RandomUnderSampler
import time

Verify pandas==1.2.4


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


# Data

In [2]:
# clone the data directory
! git clone https://github.com/ljhoh1/Dissertation_Data_2

Cloning into 'Dissertation_Data_2'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 87 (delta 36), reused 77 (delta 28), pack-reused 0
Unpacking objects: 100% (87/87), done.
Checking out files: 100% (35/35), done.


# Helper Functions

### String Cleaner

In [3]:
# transforming string features
def string_cleaner(data):
  '''
  Creating a function to remove non-numerical/alphabetical string characters and merges features
  Input:
  - data (pandas DataFrame): the input data containing at least one non-identifier column
  Output:
  - data(pandas DataFrame): the transformed dataset
  '''
  # remove special characters in all features but id and cast as lower characters
  for col in data.columns:
    if col != "id":    
      data[col] = data[col].astype(str).str.replace('[^A-Za-z0-9 ]+', '')
      data[col] = data[col].astype(str).str.lower()
      data.loc[data[col] == 'nan', [col]] = np.nan

  #merge columns into one and remove all others
  cols_data = [i for i in data.columns if i != "id"]
  data["title"] = data[cols_data].astype(str).agg(' '.join, axis=1)
  for i in cols_data:
    if i != "id" and i != "title":
      del data[i]

  return data 

### Find Training Samples

In [4]:
def find_uncertain(predictions, k = 20):
  '''
  Function for finding the most uncertain prediction
  Inputs:
  - predictions (pd.DataFrame)
  - k (int64): the number of most uncertain observations to use
  Output:
  - list_id (list): list containing the indeces of those observations identified as most uncertain
  '''
  # get match scores and compute entropy
  p = predictions["match_score"]
  predictions["entropy"] = -p * np.log(p) - (1-p) * np.log(1-p)
  # find index of most uncertain positive and negative pairs (i.e. matches and non-matches)
  to_lab_pos = predictions[predictions["match_score"] > 0.5].nlargest(10, "entropy")
  to_lab_pos.reset_index(inplace=True)
  to_lab_neg = predictions[predictions["match_score"] < 0.5].nlargest(10, "entropy")
  to_lab_neg.reset_index(inplace=True)
  # create combined list
  lis_from_series = to_lab_pos['_id'].tolist()
  lis_from_series_neg = to_lab_neg['_id'].tolist()
  list_id = [int(el) for el in lis_from_series]
  for el in lis_from_series_neg:
    list_id.append(int(el))
  del predictions["entropy"]
  return list_id

# TODO find positive and negative most certain for both functions
def find_certain(predictions, k = 20):
  '''
  Finding the predictions classified most certain, i.e. with the highest match-score
  Input:
  - predictions (pd.DataFrame): contains the observations and their respective predictions in match_score feature
  - k (int64): the number of most certain observations to use
  Output:
  - list_id (list): list containing the indeces of those observations identified as most certain
  '''
  # get match scores and compute entropy
  p = predictions["match_score"]
  predictions["entropy"] = -p * np.log(p) - (1-p) * np.log(1-p)
  # find index of most certain positive and negative pairs (i.e. matches and non-matches)
  to_lab_pos = predictions[predictions["match_score"] > 0.5].nsmallest(10, "entropy")
  to_lab_pos.reset_index(inplace=True)
  to_lab_neg = predictions[predictions["match_score"] < 0.5].nsmallest(10, "entropy")
  to_lab_neg.reset_index(inplace=True)
  # create combined list
  lis_from_series = to_lab_pos['_id'].tolist()
  lis_from_series_neg = to_lab_neg['_id'].tolist()
  list_id = [int(el) for el in lis_from_series]
  for el in lis_from_series_neg:
    list_id.append(int(el))
  del predictions["entropy"]
  return list_id

### Labelleing

In [5]:
def labelling(train, labelled, key = "idDBLP", val = "idScholar"):
  '''
  Create dictionary for checking how many matches are retained and perform labelling
  Input:
  - train (pd.DataFrame): DataFrame object of blocked data used for training
  - lbelled (pd.DataFrame): The labelled DataFrame for labelling the train data
  - key (string): which table id in blocked "train" to use as key for labelling
  - val (string): which table id in blocked "train" to use as value for labelling
  Output:
  - train (pd.DataFrame): The labelled training dataframe
  '''
  # merge id columns in one to uniquely identify
  train["id_comb"] = train["rtable_id"].astype(str) + train["ltable_id"].astype(str)
  labelled["id_comb"] = labelled[key].astype(str) + labelled[val].astype(str)
  # merge the training and labelled dataset and assign labels
  K = train.merge(labelled, on='id_comb', how='left', indicator=True)
  K.loc[(K['_merge'] == "both"), 'label'] = 1
  K.loc[(K['_merge'] == "left_only") | (K['_merge'] == "right_only"), 'label'] = 0
  # delete unused columns
  del K['_merge']
  del K[key]
  del K[val]
  del K['id_comb']

  # cleaning the data
  for col in K.columns:
    if 'id' not in col and col != "label":
      K[col] = K[col].astype(str).str.replace('[^A-Za-z0-9 ]+', '')
  
  if key == "id1":
      K['_id'] = K['_id'].astype(int)
  K['label'] = K['label'].astype(int)
  return K

### Scores

In [6]:
def scores(time, predictions, iteration, train_n, model, epochs=15):
  '''
  Report the most important metrics of performance from the predictions made by the model
  Input:
  - time (float): time the model ran for
  - predictions (pd.DataFrame): contains the output features and column "match_score", containing the prediction information
  - iteration (int64): the current iteration if applicable
  - train_n (int64): the number of samples used in training
  - model (string): the model that was used
  - epochs (int64): the number of epochs the model was trained for
  Output:
  - dict (dictionary): dictionary containing the performance information  
  '''
  # casting prediction scores as binary labels
  predictions['match_prediction'] = predictions['match_score'].apply(lambda score: 1 if score >= 0.5 else 0)
  # Reset index as Magellan requires the key to be a column in the table
  predictions.reset_index(inplace=True)
  # Update metadata in the catalog. This information can later be used by triggers to modify the labels from 
  # the learning-based matcher 
  em.set_key(predictions, '_id')
  em.set_fk_ltable(predictions, 'ltable_id')
  em.set_fk_rtable(predictions, 'rtable_id')
  # AUC-ROC metrics
  auc = sk.metrics.roc_auc_score(predictions['label'], predictions['match_score'], average = None)
  # Precision Metrics
  prec = em.eval_matches(predictions, 'label', 'match_prediction')
  # Creating dictionary of performance metrics to append to a DataFrame
  dict = {'Model':model, 'Time':time, 'Epochs':epochs,'Iteration':iteration, '#Training_samples':train_n,
                          'F1':prec['f1'], 'Precision':prec['precision'], 
                          'Recall':prec['recall'], 'AUC':auc}
  return dict

### Subsampling

In [7]:
def subsampling(cand):
  '''
  Function to subsample to avoid unbalanced data
  Input:
  - cand (pd.DataFrame): contains the blocked (and transformed) data with "label" feature
  Output:
  - cand_resampled (pd.DataFrame): contains the resampled data to avoid imbalance
  '''
  # subsampling the candidate set for better balance
  # creating the explanatory and dependent feature data
  binary_X = cand.loc[:, cand.columns != "label"]
  binary_y = cand.loc[:, cand.columns == "label"]
  # initialising the RandomUnderSampler
  rus = RandomUnderSampler(random_state=0)
  # resampling to 0.5/0.5
  X_res, y_res = rus.fit_resample(binary_X, binary_y)
  res_labelled = pd.DataFrame(X_res)
  res_labelled.columns = cand.columns[0:-1]
  # take the index of resampled observations to subsample from the original
  sel_idx = list(res_labelled._id)
  cand_resampled = cand[cand["_id"].isin(sel_idx)]
  return cand_resampled

### ID Selection

In [90]:
def id_selection(predictions = None, cand = None, train_idx = None, pred_idx = None, 
                 val_idx = None, train_type = None, C = None, cora_flag = None, labelled = None):
  '''
  Function that performs the selection of IDs either randomly or according to active learning
  Inputs:
  - predictions (pd.DataFrame): contains the output features and column "match_score", containing the prediction information
  - cand (pd.DataFrame): contains the blocked (and transformed) data with "label" feature
  - train_idx (list): list containing the indeces of training observations
  - pred_idx (list): list containing the indeces of observations used for prediction
  - val_idx (list): list containing the indeces of observations used for validation
  - train_type (str): flag for whether to apply random selection on all iterations
  - C (pd.DataFrame): contains the blocked (and transformed) source data without "label" feature
  - cora_flag (bool): flag indicating that Cora data is used
  - train_n (int64): the number of samples used in training
    Output:
  - train_idx (list): list containing the updated indeces of training observations
  - pred_idx (list): list containing the updated indeces of observations used for prediction
  - val_idx (list): list containing the updated indeces of observations used for validation
  - cand (pd.DataFrame): contains the blocked (and transformed) data with "label" feature
  '''
  if train_type == "trans_act" and train_idx == None:
    if cora_flag == False:
        cand = C
        cand = labelling(cand, labelled)
    else:
        cand = C
    print("a")
    train_idx = []
    pred_idx = list(np.arange(cand.shape[0]))
    uncertain_id = find_uncertain(predictions, k = 20)
    [train_idx.append(id) for id in uncertain_id]
    certain_id = find_certain(predictions, k = 20)
    # assigning the predicted value to candidate set
    cand.loc[certain_id]['label'] = round(predictions['match_score'])
    [train_idx.append(id) for id in certain_id]
    pred_idx = list(pred_idx)
    for id in train_idx:
      if id in pred_idx:
        pred_idx.remove(id)

    val_idx = np.random.choice(pred_idx, 1000, replace=False)
    val = cand.iloc[val_idx]
    path_val = os.path.join('.', 'Dissertation_Data_2', 'val.csv')
    val.to_csv(path_val, index=False) 

    for id in val_idx:
      if id in pred_idx:
        pred_idx.remove(id)
        
  if train_idx == None:
      if cora_flag == False:
        cand = C
        cand = labelling(cand, labelled)
      else:
        cand = C

      pred_idx = list(np.arange(cand.shape[0]))
      train_idx = list(np.random.choice(range(cand.shape[0]), 200, replace=False))
      for id in train_idx:
        pred_idx.remove(id)

      val_idx = np.random.choice(pred_idx, 1000, replace=False)
      val = cand.iloc[val_idx]
      path_val = os.path.join('.', 'Dissertation_Data_2', 'val.csv')
      val.to_csv(path_val, index=False) 
      for id in val_idx:
        if id in pred_idx:
          pred_idx.remove(id)

  elif train_type == "rand":
    add_train_idx = list(np.random.choice(range(predictions.shape[0]), 40, replace=False))
    [train_idx.append(id) for id in add_train_idx]

    pred_idx = list(pred_idx)
    for id in train_idx:
      if id in pred_idx:
        pred_idx.remove(id)
    
  else:
    uncertain_id = find_uncertain(predictions, k = 20)
    [train_idx.append(id) for id in uncertain_id]
    certain_id = find_certain(predictions, k = 20)
    # assigning the predicted value to candidate set
    cand.loc[certain_id]['label'] = round(predictions['match_score'])
    [train_idx.append(id) for id in certain_id]
    pred_idx = list(pred_idx)
    for id in train_idx:
      if id in pred_idx:
        pred_idx.remove(id)

  train = cand.iloc[train_idx]
  pred = cand.iloc[pred_idx]
  path_train = os.path.join('.', 'Dissertation_Data_2', 'train.csv')
  path_pred = os.path.join('.', 'Dissertation_Data_2', 'test.csv')
  train.to_csv(path_train, index=False)
  pred.to_csv(path_pred, index=False)

  return train_idx, val_idx, pred_idx, cand

### Active Learning

In [101]:
# active learning algorithm
def active_learning(source = None, iter = 5, executionTime = 0, train_type = None, test_init = None,dict_trans = None, model_n = None,
                    model_path = "hybrid_model.pth", cache = 'train_cache_DS.pth', cora_flag = False, label = None):
  '''
  Function that performs the active learning
  Inputs:
  - source (pd.DataFrame): contains the blocked features for matching and prediction
  - iter (int64): the number of iterations to perform
  - train_type (str): string flag indicating whether training is random, active or transfer-active
  - cache (str): string indicating filename for saving data
  - cora_flag (bool): flag indicating that Cora data is used
    Output:
  - prec_matrix (pd.DataFrame): data frame with performance measures from every iteration
  '''
  if train_type == "trans_act":
    model = dm.MatchingModel(attr_summarizer='hybrid')
    model.load_state(model_path)

    train, val, pred = dm.data.process(
      path=os.path.join('.', 'Dissertation_Data_2'),
      cache=cache,
      train='train.csv',
      validation="val.csv",
      test=test_init,
      use_magellan_convention=True,
      ignore_columns=('ltable_id', 'rtable_id', '_id'))

  else:
    # selecting those rows to be used
    train_idx, val_idx, pred_idx, cand = id_selection(C = source, cora_flag = cora_flag, train_type = train_type, labelled = label)

    # processing the data
    train, val, pred = dm.data.process(
        path=os.path.join('.', 'Dissertation_Data_2'),
        cache=cache,
        train='train.csv',
        validation="val.csv",
        test='test.csv',
        use_magellan_convention=True,
        ignore_columns=('ltable_id', 'rtable_id', '_id'))

    # Create a hybrid model.
    model = dm.MatchingModel(attr_summarizer='hybrid')
    model.initialize(train)  # Initilization

    # Train ing model on 15 epochs, batch size of 16, positive-to-negative 
    # ratio to be 1:1. We save the best model (with the 
    # highest F1 score on the validation set) to 'hybrid_model.pth'.
    startTime = time.time()
    model.run_train(
        train,
        val,
        epochs=15,
        batch_size=16,
        best_save_path=model_path)
    executionTime = (time.time() - startTime)/60

  # Evaluate the accuracy on the test data.
  predictions = model.run_prediction(pred, output_attributes=list(pred.get_raw_table().columns))
  # getting predictions and compute performance metrics
  prec_matrix = pd.DataFrame(columns=['Iteration', '#Training_samples', 'F1', 'Precision', 'Recall', 'AUC'])
  
  if train_type == "trans_act":
    dict_trans = model.attr_summarizers.state_dict()
    score = scores(executionTime, predictions, 0, 0, model = model_n)
    prec_matrix = prec_matrix.append(score, ignore_index=True)
    # selecting those rows to be used
  elif cora_flag:
    prediction_scores = model.run_prediction(test, output_attributes=list(test.get_raw_table().columns))
    # Precision Metrics
    score = scores(executionTime, prediction_scores, 0, len(train_idx), model = model_n)
    prec_matrix = prec_matrix.append(score, ignore_index=True)
  else:
    # Precision Metrics
    score = scores(executionTime, predictions, 0, len(train_idx), model = model_n)
    prec_matrix = prec_matrix.append(score, ignore_index=True)

  # perform active learning on the specified iterations
  for i in range(iter):
      # model for candidate set preprocessing.
      if train_type == "trans_act" and i == 0:
        train_idx, val_idx, pred_idx, cand = id_selection(predictions, C=source, labelled = label,
                                                        train_type = train_type, cora_flag = cora_flag)
      elif train_type == "trans_act" and i > 0: 
        train_idx, val_idx, pred_idx, cand = id_selection(predictions, cand, train_idx, pred_idx, val_idx, 
                                                        train_type = train_type, cora_flag = cora_flag)
      else:   
        train_idx, val_idx, pred_idx, cand = id_selection(predictions, cand, train_idx, pred_idx, val_idx, 
                                                        train_type = train_type, cora_flag = cora_flag)
      # processing the data
      train, val, pred = dm.data.process(
        path=os.path.join('.', 'Dissertation_Data_2'),
        cache=cache,
        train='train.csv',
        validation="val.csv",
        test='test.csv',
        use_magellan_convention=True,
        ignore_columns=('ltable_id', 'rtable_id', '_id'))     
      # Load the model.
      model = dm.MatchingModel(attr_summarizer='hybrid')
      
      if train_type == "trans_act":
        model.initialize(train)
        model.attr_summarizers.load_state_dict(dict_trans)
      else:
        model.load_state(model_path)
      # Load the candidate set. Note that the trained model is an input parameter as we need to trained 
      startTime = time.time()
      model.run_train(
          train,
          val,
          epochs=10,
          batch_size=16,
          best_save_path=model_path)
      executionTime = (time.time() - startTime)/60      
      predictions = model.run_prediction(pred, output_attributes=list(pred.get_raw_table().columns))
      if train_type == "trans_act":
        dict_trans = model.attr_summarizers.state_dict()
      print(f"Iteration:{i + 1} with training samples {len(train_idx)}")
      print(f"Iteration:{i + 1} with prediction samples {len(pred_idx)}")
      if cora_flag:
        prediction_scores = model.run_prediction(test, output_attributes=list(test.get_raw_table().columns))
        # Precision Metrics
        score = scores(executionTime, prediction_scores, 0, len(train_idx), model = model_n)
        prec_matrix = prec_matrix.append(score, ignore_index=True)
      else:
        # Precision Metrics
        score = scores(executionTime, predictions, 0, len(train_idx), model = model_n)
        prec_matrix = prec_matrix.append(score, ignore_index=True)
  return prec_matrix, predictions

# Active Learning

## 1. DBLP-Scholar

### Data

In [10]:
 # The path to the labeled data file.
path_Match = os.path.join('.', 'Dissertation_Data_2', 'DBLP-Scholar_Perfect_Mapping.csv')
labelled_DS = em.read_csv_metadata(path_Match)
# The path to the two input tables.
path_Scholar = os.path.join('.', 'Dissertation_Data_2', 'Scholar_data.csv')
path_DBLP = os.path.join('.', 'Dissertation_Data_2', 'DBLP1_data.csv')
# Load the Scholar data table
Scholar = em.read_csv_metadata(path_Scholar, key='id')
# Transforming the title column cells
Scholar['title'] = Scholar['title'].str.lower()
Scholar['title'] = Scholar['title'].str.replace('[^A-Za-z0-9 ]+', '')
Scholar.head()
# DBLP data
DBLP = em.read_csv_metadata(path_DBLP, key='id')
DBLP['title'] = DBLP['title'].str.replace('[^A-Za-z0-9 ]+', '')
DBLP['title'] = DBLP['title'].str.lower()
DBLP.head()
# BLOCKING
# Get tokenizers and similarity function
block_t = em.get_tokenizers_for_blocking()
block_s = em.get_sim_funs_for_blocking()
# Get attributes
atypes1 = em.get_attr_types(Scholar)
atypes2 = em.get_attr_types(DBLP)
# Get correspondence
block_c = em.get_attr_corres(Scholar, DBLP)
#Get Features
block_f = em.get_features(Scholar, DBLP, atypes1, atypes2, block_c, block_t, block_s)
# Creating a rule based blocker
rb = em.RuleBasedBlocker()
# 0.41 achieves 49% balance negative to positive
rule = ['title_title_jac_qgm_3_qgm_3(ltuple, rtuple) < 0.41']
rb.add_rule(rule, feature_table=block_f)
C = rb.block_tables(Scholar, DBLP,
                   l_output_attrs=['title', 'authors', 'venue', 'year'], 
                   r_output_attrs=['title', 'authors', 'venue', 'year'], 
                   n_jobs=1)

# subsample from blocked dataset and reset index for active learning model
#C = C.sample(frac=0.5, random_state=1)
C = C.reset_index()
C['_id'] = np.arange(C.shape[0])
C = C.drop(['index'], axis = 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  from ipykernel import kernelapp as app
0% [##############################] 100% | ETA: 00:00:00

Finding pairs with missing value...



Total time elapsed: 00:00:35


### Algorithm

In [11]:
# active learning algorithm
mat_act_DS, pred_act_ds = active_learning(source = C, iter = 20, model_n = 'activeDS', label = labelled_DS)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: FutureWarning: The default value of regex will change from True to False in a future version.

Reading and processing data from "./Dissertation_Data_2/train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "./Dissertation_Data_2/val.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "./Dissertation_Data_2/test.csv"
0% [############################# ] 100% | ETA: 00:00:00INFO:deepmatcher.data.field:Downloading vectors from https://drive.google.com/uc?export=download&id=1Vih8gAmgBnuYDxfblbT94P6WjB7s1ZSh to /root/.vector_cache/wiki.en.bin
INFO:deepmatcher.data.field:Unable to fetch cached English Word Embeddings from https://drive.google.com/uc?export=download&id=1Vih8gAmgBnuYDxfblbT94P6WjB7s1ZSh
/usr/local/lib/python3.7/dist-packages/deepmatcher/data/field.py:79: ResourceWarning: unclosed <ssl.SSLSocket fd=64, family=AddressFamily.AF_I

* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Finished Epoch 1 || Run Time:    1.6 | Load Time:    0.2 || F1:  52.34 | Prec:  50.45 | Rec:  54.37 || Ex/s: 109.45

===>  EVAL Epoch 1


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


...

Finished Epoch 10 || Run Time:    6.3 | Load Time:    1.1 || F1:  99.80 | Prec:  99.80 | Rec:  99.80 || Ex/s: 135.63

===>  EVAL Epoch 10


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 10 || Run Time:    3.2 | Load Time:    1.1 || F1:  94.54 | Prec:  94.54 | Rec:  94.54 || Ex/s: 233.08

---------------------

Loading best model...
Training done.
===>  PREDICT Epoch 4


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:23


Finished Epoch 4 || Run Time:   14.9 | Load Time:    8.9 || F1:  94.59 | Prec:  94.40 | Rec:  94.78 || Ex/s: 365.65

Iteration:20 with training samples 1000
Iteration:20 with prediction samples 8693


In [12]:
# active learning algorithm
mat_act_DS_rand, pred_act_rand_ds = active_learning(source = C, iter = 20, train_type = "rand", model_n = 'activeDS_rand', label = labelled_DS)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: FutureWarning: The default value of regex will change from True to False in a future version.

Reading and processing data from "./Dissertation_Data_2/train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "./Dissertation_Data_2/val.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "./Dissertation_Data_2/test.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:    1.3 | Load Time:    0.2 || F1:  59.30 | Prec:  66.23 | Rec:  53.68 || Ex/s: 129.83

===>  EVAL Epoch 1


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


...

Finished Epoch 10 || Run Time:    6.6 | Load Time:    1.1 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s: 129.32

===>  EVAL Epoch 10


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 10 || Run Time:    3.3 | Load Time:    1.1 || F1:  94.51 | Prec:  94.15 | Rec:  94.87 || Ex/s: 229.92

---------------------

Loading best model...
Training done.
===>  PREDICT Epoch 2


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:23


Finished Epoch 2 || Run Time:   15.2 | Load Time:    8.8 || F1:  92.71 | Prec:  90.32 | Rec:  95.22 || Ex/s: 367.49

Iteration:20 with training samples 1000
Iteration:20 with prediction samples 8815


In [13]:
pred_act_ds['match_prediction'] = pred_act_ds['match_score'].apply(lambda score: 1 if score >= 0.5 else 0)
path_pred_act_ds = os.path.join('.', 'Dissertation_Data_2', 'pred_act_ds.csv')
pred_act_ds.to_csv(path_pred_act_ds)

In [14]:
pred_act_rand_ds['match_prediction'] = pred_act_rand_ds['match_score'].apply(lambda score: 1 if score >= 0.5 else 0)
path_pred_act_rand_ds = os.path.join('.', 'Dissertation_Data_2', 'pred_act_ds_rand.csv')
pred_act_rand_ds.to_csv(path_pred_act_rand_ds)

## 2. Cora-Cora

### Data

In [15]:
# load data for cora active learning
path_Match_Cora = os.path.join('.', 'Dissertation_Data_2', 'cora_duplicates_data.tsv')
labelled_Cora = em.read_csv_metadata(path_Match_Cora)
path_Cora = os.path.join('.', 'Dissertation_Data_2', 'cora_data.tsv')
csv_table=pd.read_table(path_Cora,sep='\t')
csv_table.to_csv(os.path.join('.', 'Dissertation_Data_2', 'cora_data.csv'),index=False)
Cora = em.read_csv_metadata(os.path.join('.', 'Dissertation_Data_2', 'cora_data.csv'), key='id')
# transform Cora data
for col in Cora.columns:
  Cora[col] = Cora[col].astype(str).str.replace('[^A-Za-z0-9 ]+', '')
  Cora[col] = Cora[col].astype(str).str.lower()
  Cora.loc[Cora[col] == 'nan', [col]] = np.nan

Cora['date'] = Cora['date'].astype(str).str.replace('[^0-9]+', '')
Cora.year.fillna(Cora.date, inplace=True)
Cora.loc[Cora['year'] == '', ['year']] = np.nan
del Cora['date']

Cora.rename(columns = {'type':'new_type'}, inplace = True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [16]:
# The path to the labeled data file.
csv_table=pd.read_table(path_Match_Cora, sep='\t')
csv_table.to_csv(os.path.join('.', 'Dissertation_Data_2', 'cora_labelled.csv'),index=False)
cora_labelled = em.read_csv_metadata(os.path.join('.', 'Dissertation_Data_2', 'cora_labelled.csv'))

In [17]:
# Blocking - Cora/Cora
# Get tokenizers and similarity function
block_t = em.get_tokenizers_for_blocking()
block_s = em.get_sim_funs_for_blocking()
# Get attributes
atypes1 = em.get_attr_types(Cora)
atypes2 = em.get_attr_types(Cora)
# Get correspondence
block_c = em.get_attr_corres(Cora, Cora)
#Get Features
block_f = em.get_features(Cora, Cora, atypes1, atypes2, block_c, block_t, block_s)
cols_cora = [i for i in Cora.columns]
# Creating a rule based blocker
rb = em.RuleBasedBlocker()
cols_cora = [i for i in Cora.columns]
rule = ['title_title_jac_qgm_3_qgm_3(ltuple, rtuple) < 0.35']
rb.add_rule(rule, feature_table=block_f)
K3 = rb.block_tables(Cora, Cora,
                   l_output_attrs=cols_cora, 
                   r_output_attrs=cols_cora,
                   n_jobs=1)
# Creating a rule based blocker
rb = em.RuleBasedBlocker()
rule = ['authors_authors_jac_qgm_3_qgm_3(ltuple, rtuple) < 0.1']
rb.add_rule(rule, feature_table=block_f)
K4 = rb.block_candset(K3, n_jobs=1)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Finding pairs with missing value...


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:54


In [18]:
cols = list(cora_labelled.columns)
a, b = cols.index('id1'), cols.index('id2')
cols[b], cols[a] = cols[a], cols[b]
cor_lab_rev = cora_labelled[cols]
cor_lab_rev.columns = ['id1', 'id2']
cora_labelled = cora_labelled.append(cor_lab_rev)

# labelling and subsampling data
cand = labelling(K4, cora_labelled, key = "id1", val = "id2" )
print("Proportion of matches retained after blocking:", round(sum(cand['label']) / cora_labelled.shape[0], 2))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: FutureWarning: The default value of regex will change from True to False in a future version.


Proportion of matches retained after blocking: 1.0


In [19]:
cand = cand.reset_index()
cand['_id'] = np.arange(cand.shape[0])
cand = cand.drop(['index'], axis = 1)

In [20]:
sum(cand['label'])

129031

In [103]:
cand.to_csv(os.path.join('.', 'Dissertation_Data_2', 'pred_Cora.csv'),index=False)
# The directory where the data splits will be saved.
split_path = os.path.join('.', 'Dissertation_Data_2')
# Split labeled data into train, valid, and test csv files to disk, with the split ratio of 3:1:1.
dm.data.split(cand, split_path, 'train_cor.csv', 'val_cor.csv', 'test_cor.csv',
              [3, 1, 1])

In [22]:
train_tosub = em.read_csv_metadata(os.path.join('.', 'Dissertation_Data_2', 'train_cor.csv'))

cand_resampled = train_tosub.sample(frac=0.6, random_state=1)
cand_resampled.to_csv(os.path.join('.', 'Dissertation_Data_2', 'train_cor.csv'),index=False)

In [23]:
cand_resampled.shape

(82194, 34)

In [24]:
val_tosub = em.read_csv_metadata(os.path.join('.', 'Dissertation_Data_2', 'val_cor.csv'))

val_resampled = val_tosub.sample(frac=0.5, random_state=1)
val_resampled.to_csv(os.path.join('.', 'Dissertation_Data_2', 'val_cor.csv'),index=False)

In [25]:
# Load the training data files from the disk. Ignore the "left_id" and "right_id" 
# columns for data preprocessing.
# The 'use_magellan_convention' parameter asks deepmatcher to use Magellan's 
# naming convention for the left and right table column name prefixes 
# ("ltable_", and "rtable_"), and also to consider "_id" as the ID column.
train_act_cor, validation_act_cor, test = dm.data.process(
    path=os.path.join('.', 'Dissertation_Data_2'),
    cache='train_cache.pth',
    train='train_cor.csv',
    validation='val_cor.csv',
    test='test_cor.csv',
    use_magellan_convention=True,
    ignore_columns=('ltable_id', 'rtable_id'))


Reading and processing data from "./Dissertation_Data_2/train_cor.csv"
0% [##############################] 100% | ETA: 00:00:00
Reading and processing data from "./Dissertation_Data_2/val_cor.csv"
0% [##############################] 100% | ETA: 00:00:00
Reading and processing data from "./Dissertation_Data_2/test_cor.csv"
0% [##############################] 100% | ETA: 00:00:00
Building vocabulary
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:12

Computing principal components
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:13


In [26]:
cand_resampled = cand_resampled.reset_index()
cand_resampled['_id'] = np.arange(cand_resampled.shape[0])
cand_resampled = cand_resampled.drop(['index'], axis = 1)

### Algorithm

In [27]:
# active learning algorithm
mat_act_cora, pred_act_cor = active_learning(source = cand_resampled, iter = 5, cache = 'train_cache_Cora.pth', cora_flag = True, model_n = 'activeCora')


Reading and processing data from "./Dissertation_Data_2/train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "./Dissertation_Data_2/val.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "./Dissertation_Data_2/test.csv"
0% [##############################] 100% | ETA: 00:00:00
Building vocabulary
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/l

* Number of trainable parameters: 33136817
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 1 || Run Time:    4.8 | Load Time:    0.5 || F1:  71.15 | Prec:  66.67 | Rec:  76.27 || Ex/s:  37.87

===>  EVAL Epoch 1


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:14


...

Finished Epoch 10 || Run Time:   12.1 | Load Time:    2.8 || F1:  97.83 | Prec:  97.48 | Rec:  98.19 || Ex/s:  66.98

---------------------

Loading best model...
Training done.
===>  PREDICT Epoch 3


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:12:28


Finished Epoch 3 || Run Time:  509.6 | Load Time:  239.1 || F1:  98.46 | Prec:  97.44 | Rec:  99.49 || Ex/s: 107.92

Iteration:5 with training samples 400
Iteration:5 with prediction samples 80794
===>  PREDICT Epoch 3


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:07:05


Finished Epoch 3 || Run Time:  289.6 | Load Time:  136.9 || F1:  98.42 | Prec:  97.26 | Rec:  99.62 || Ex/s: 107.05



In [28]:
# active learning algorithm
mat_act_cora_rand, pred_act_cor_rand = active_learning(source = cand_resampled, iter = 5, cache = 'train_cache_Cora.pth', cora_flag = True, train_type = "rand", model_n = 'activeCora_rand')


Reading and processing data from "./Dissertation_Data_2/train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "./Dissertation_Data_2/val.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "./Dissertation_Data_2/test.csv"
0% [##############################] 100% | ETA: 00:00:00
Building vocabulary
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/l

* Number of trainable parameters: 33136817
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 1 || Run Time:    5.2 | Load Time:    0.6 || F1:  75.00 | Prec:  70.00 | Rec:  80.77 || Ex/s:  34.64

===>  EVAL Epoch 1


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:14


...

Finished Epoch 10 || Run Time:   11.9 | Load Time:    2.7 || F1:  97.52 | Prec:  95.48 | Rec:  99.65 || Ex/s:  68.63

---------------------

Loading best model...
Training done.
===>  PREDICT Epoch 2


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:12:32


Finished Epoch 2 || Run Time:  512.7 | Load Time:  239.5 || F1:  97.68 | Prec:  96.08 | Rec:  99.34 || Ex/s: 107.42

Iteration:5 with training samples 400
Iteration:5 with prediction samples 80797
===>  PREDICT Epoch 2


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:07:07


Finished Epoch 2 || Run Time:  290.8 | Load Time:  136.9 || F1:  96.91 | Prec:  94.75 | Rec:  99.17 || Ex/s: 106.75



In [29]:
pred_act_cor['match_prediction'] = pred_act_cor['match_score'].apply(lambda score: 1 if score >= 0.5 else 0)
path_pred_act_cor = os.path.join('.', 'Dissertation_Data_2', 'pred_act_cor.csv')
pred_act_cor.to_csv(path_pred_act_cor)

In [30]:
pred_act_cor_rand['match_prediction'] = pred_act_cor_rand['match_score'].apply(lambda score: 1 if score >= 0.5 else 0)
path_pred_act_rand_cor = os.path.join('.', 'Dissertation_Data_2', 'pred_act_cor_rand.csv')
pred_act_cor_rand.to_csv(path_pred_act_rand_cor)

# Transfer Active


## Transfer

### Cora base

In [31]:
!cp "/content/Dissertation_Data_2/hybrid_Trans_Cor.pth" "/content"
!cp "/content/Dissertation_Data_2/hybrid_Trans_DS.pth" "/content"

In [32]:
# Loading labelled and non-labelled Cora data
Cora = em.read_csv_metadata(os.path.join('.', 'Dissertation_Data_2', 'cora_data.csv'), key='id')
cora_labelled = em.read_csv_metadata(os.path.join('.', 'Dissertation_Data_2', 'cora_labelled.csv'), 
                         ltable=Cora, rtable=Cora, 
                         fk_ltable='id2', fk_rtable='id1')

# cleaning the date feature in Cora and applying string cleaner to all data
for col in Cora.columns:
  Cora[col] = Cora[col].astype(str).str.replace('[^A-Za-z0-9 ]+', '')
  Cora[col] = Cora[col].astype(str).str.lower()

Cora['date'] = Cora['date'].astype(str).str.replace('[^0-9]+', '')
Cora.year.fillna(Cora.date, inplace=True)
Cora.loc[Cora['year'] == '', ['year']] = np.nan
del Cora['date']

Cora.rename(columns = {'type':'new_type'}, inplace = True)

Cora = string_cleaner(Cora)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]


In [33]:
# feature blocking for Cora 
# Get tokenizers and similarity function
block_t = em.get_tokenizers_for_blocking()
block_s = em.get_sim_funs_for_blocking()
# Get attributes
atypes1 = em.get_attr_types(Cora)
atypes2 = em.get_attr_types(Cora)
# Get correspondence
block_c = em.get_attr_corres(Cora, Cora)
#Get Features
block_f = em.get_features(Cora, Cora, atypes1, atypes2, block_c, block_t, block_s)

In [34]:
# Creating a rule based blocker with threshold 0.4
rb = em.RuleBasedBlocker()
cols_cora = [i for i in Cora.columns]
rule = ['title_title_jac_qgm_3_qgm_3(ltuple, rtuple) < 0.35']
rb.add_rule(rule, feature_table=block_f)
K1 = rb.block_tables(Cora, Cora,
                   l_output_attrs=cols_cora, 
                   r_output_attrs=cols_cora,
                   n_jobs=1)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finding pairs with missing value...


In [35]:
cols = list(cora_labelled.columns)
a, b = cols.index('id1'), cols.index('id2')
cols[b], cols[a] = cols[a], cols[b]
cor_lab_rev = cora_labelled[cols]
cor_lab_rev.columns = ['id1', 'id2']
cora_labelled = cora_labelled.append(cor_lab_rev)

In [36]:
# Labelling the blocked data set
cand = labelling(K1, cora_labelled, key = "id1", val = "id2")
print("Proportion of matches retained after blocking:", round(sum(cand['label']) / cora_labelled.shape[0], 2))

# subsampling the candidate set for better balance
for col in cand.columns:
    if col != "_id" and col != "label":
      cand[col] = cand[col].astype(str).str.replace('[^A-Za-z0-9 ]+', '')
    else:
      cand[col] = cand[col].astype(int)

#cand = cand.sample(frac=0.5, random_state=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: FutureWarning: The default value of regex will change from True to False in a future version.


Proportion of matches retained after blocking: 1.0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [ ]:
# The directory where the data splits will be saved.
split_path = os.path.join('.', 'Dissertation_Data_2')
# Split labeled data into train, valid, and test csv files to disk, with the split ratio of 3:1:1.
dm.data.split(cand, split_path, 'train_trans_cor.csv', 'valid_trans_cor.csv', 'test_trans_cor.csv',
              [3, 1, 1])

In [38]:
print("Proportion of matches retained after blocking:", round(sum(cand['label']) / cand.shape[0], 2))

Proportion of matches retained after blocking: 0.7


In [ ]:
# Load the training data files from the disk. Ignore the "left_id" and "right_id" 
# columns for data preprocessing.
# The 'use_magellan_convention' parameter asks deepmatcher to use Magellan's 
# naming convention for the left and right table column name prefixes 
# ("ltable_", and "rtable_"), and also to consider "_id" as the ID column.
train, validation, test = dm.data.process(
    path=os.path.join('.', 'Dissertation_Data_2'),
    cache='train_cache.pth',
    train='train_trans_cor.csv',
    validation='valid_trans_cor.csv',
    test='test_trans_cor.csv',
    use_magellan_convention=True,
    ignore_columns=('ltable_id', 'rtable_id'))

In [40]:
# Create a hybrid model.
model_cor = dm.MatchingModel(attr_summarizer='hybrid')
model_cor.initialize(train)  # Initilization
startTime = time.time()
# Train ing model on 10 epochs, batch size of 16, positive-to-negative 
# ratio to be 1:1. We save the best model (with the 
# highest F1 score on the validation set) to 'sifDiff_model.pth'.
model_cor.run_train(
    train,
    validation,
    epochs=10,
    batch_size=128,
    best_save_path='hybrid_Trans_Cor.pth')
execTime_cor = (time.time() - startTime)/60
# Evaluate the accuracy on the test data.
#model_cor.run_eval(test)

* Number of trainable parameters: 2798703
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:  109.6 | Load Time:  187.2 || F1:  97.69 | Prec:  96.52 | Rec:  98.89 || Ex/s: 370.94

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:01:17


Finished Epoch 1 || Run Time:   14.5 | Load Time:   63.3 || F1:  99.19 | Prec:  98.54 | Rec:  99.85 || Ex/s: 471.77

* Best F1: tensor(99.1881, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

Finished Epoch 9 || Run Time:   14.4 | Load Time:   62.7 || F1:  99.28 | Prec:  99.16 | Rec:  99.40 || Ex/s: 475.66

---------------------

===>  TRAIN Epoch 10


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:04:56


Finished Epoch 10 || Run Time:  109.4 | Load Time:  187.4 || F1:  99.49 | Prec:  99.23 | Rec:  99.75 || Ex/s: 371.01

===>  EVAL Epoch 10


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:01:17


Finished Epoch 10 || Run Time:   14.5 | Load Time:   63.2 || F1:  99.28 | Prec:  99.15 | Rec:  99.41 || Ex/s: 472.24

---------------------

Loading best model...
Training done.


In [41]:
dict_trans_cor = model_cor.attr_summarizers.state_dict()

### DS Base

In [42]:
# The path to the labeled data file.
path_TransAct_DS = os.path.join('.', 'Dissertation_Data_2', 'pred_DS.csv')
transAct_DS = em.read_csv_metadata(path_TransAct_DS)
# The path to the labeled data file.
path_TransAct_Cora = os.path.join('.', 'Dissertation_Data_2', 'pred_Cora.csv')
transAct_Cora = em.read_csv_metadata(path_TransAct_Cora)
del transAct_DS['label']

In [43]:
Scholar = string_cleaner(Scholar)
DBLP = string_cleaner(DBLP)
# blocking for predictions on other data set
# Get tokenizers and similarity function
block_t = em.get_tokenizers_for_blocking()
block_s = em.get_sim_funs_for_blocking()
# Get attributes
atypes1 = em.get_attr_types(Scholar)
atypes2 = em.get_attr_types(DBLP)
# Get correspondence
block_c = em.get_attr_corres(Scholar, DBLP)
#Get Features
block_f = em.get_features(Scholar, DBLP, atypes1, atypes2, block_c, block_t, block_s)
# Creating a rule based blocker for prediction
rb = em.RuleBasedBlocker()
# 0.3 achieves 49% balance negative to positive
rule = ['title_title_jac_qgm_3_qgm_3(ltuple, rtuple) < 0.29']
rb.add_rule(rule, feature_table=block_f)
C = rb.block_tables(Scholar, DBLP,
                   l_output_attrs=['title'], 
                   r_output_attrs=['title'], 
                   n_jobs=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]
0% [##############################] 100% | ETA: 00:00:00

Finding pairs with missing value...



Total time elapsed: 00:02:23


In [ ]:
# labelling data for prediction
pred = labelling(C, labelled_DS, key = "idDBLP", val = "idScholar")
print("Proportion of matches retained after blocking:", round(sum(pred['label']) / labelled_DS.shape[0], 2))

pred.to_csv(os.path.join('.', 'Dissertation_Data_2', 'pred_DS.csv'),index=False)
# The directory where the data splits will be saved.
split_path = os.path.join('.', 'Dissertation_Data_2')
# Split labeled data into train, valid, and test csv files to disk, with the split ratio of 3:1:1.
dm.data.split(pred, split_path, 'train_trans_DS.csv', 'valid_trans_DS.csv', 'test_trans_DS.csv',
              [3, 1, 1])

In [ ]:
# processing the data for prediction
train_DS, validation_DS, test_DS = dm.data.process(
    path=os.path.join('.', 'Dissertation_Data_2'),
    cache='train_trans_DS_cache.pth',
    train='train_trans_DS.csv',
    validation='valid_trans_DS.csv',
    test='test_trans_DS.csv',
    use_magellan_convention=True,
    ignore_columns=('ltable_id', 'rtable_id'))

train_DS, validation_DS, pred_DS = dm.data.process(
    path=os.path.join('.', 'Dissertation_Data_2'),
    cache='train_trans_DS2_cache.pth',
    train='train_trans_DS.csv',
    validation='valid_trans_DS.csv',
    test='pred_DS.csv',
    use_magellan_convention=True,
    ignore_columns=('ltable_id', 'rtable_id'))

In [ ]:
# Create a hybrid model for DBLP-Scholar data
model_DS = dm.MatchingModel(attr_summarizer='hybrid')
model_DS.initialize(train_DS)  # Initilization

# Train ing model on 15 epochs, batch size of 16, positive-to-negative 
# ratio to be 1:1. We save the best model (with the 
# highest F1 score on the validation set) to 'sifDiff_model.pth'.
startTime = time.time()
model_DS.run_train(
    train_DS,
    validation_DS,
    epochs=15,
    batch_size=16,
    best_save_path='hybrid_Trans_DS.pth')
execTime_DS = (time.time() - startTime)/60
# Evaluate the accuracy on the test data.
model_DS.run_eval(test_DS)

In [47]:
dict_trans_DS = model_DS.attr_summarizers.state_dict()

## Active

In [ ]:
C = C.reset_index()
C['_id'] = np.arange(C.shape[0])
C = C.drop(['index'], axis = 1)

In [97]:
 # The path to the labeled data file.
mat_trans_DS, pred_transAct_ds = active_learning(source = C, iter = 20, executionTime = execTime_DS, train_type = "trans_act", test_init = 'pred_DS.csv', dict_trans = dict_trans_cor, model_path = 'hybrid_Trans_Cor.pth', cache = 'train_cache_transActDS.pth', model_n = 'TransActOnDS', label = labelled_DS)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


===>  PREDICT Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:16
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: FutureWarning: The default value of regex will change from True to False in a future version.


Finished Epoch 1 || Run Time:    7.8 | Load Time:    9.0 || F1:  83.66 | Prec:  85.93 | Rec:  81.50 || Ex/s: 769.31

a



Reading and processing data from "./Dissertation_Data_2/train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "./Dissertation_Data_2/val.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "./Dissertation_Data_2/test.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


...

Finished Epoch 10 || Run Time:    1.8 | Load Time:    0.7 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s: 342.18

===>  EVAL Epoch 10


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Finished Epoch 10 || Run Time:    1.0 | Load Time:    0.8 || F1:  88.29 | Prec:  90.74 | Rec:  85.96 || Ex/s: 542.63

---------------------

Loading best model...
Training done.
===>  PREDICT Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:14


Finished Epoch 1 || Run Time:    6.4 | Load Time:    7.8 || F1:  88.91 | Prec:  86.76 | Rec:  91.16 || Ex/s: 787.01

Iteration:20 with training samples 840
Iteration:20 with prediction samples 11174


In [98]:
cand = cand.reset_index()
cand['_id'] = np.arange(cand.shape[0])
cand = cand.drop(['index'], axis = 1)

In [104]:
Mat_trans_cora, pred_transAct_cor = active_learning(source = cand, iter = 5, executionTime = execTime_cor, train_type = "trans_act", test_init = 'pred_Cora.csv', dict_trans = dict_trans_DS, model_path = 'hybrid_Trans_DS.pth', cache = 'train_cache_transActCora.pth', cora_flag = True, model_n = 'TransActOnCora')

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "

Reading and processing data from "./Dissertation_Data_2/train.csv"
0% [############################# ] 100% 

===>  PREDICT Epoch 10


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 10 || Run Time:  114.4 | Load Time:  157.1 || F1:  96.31 | Prec:  95.42 | Rec:  97.22 || Ex/s: 675.76

a



Reading and processing data from "./Dissertation_Data_2/train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "./Dissertation_Data_2/val.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "./Dissertation_Data_2/test.csv"
0% [##############################] 100% | ETA: 00:00:00
Building vocabulary
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


...

Finished Epoch 10 || Run Time:    1.1 | Load Time:    1.3 || F1:  97.37 | Prec:  95.28 | Rec:  99.55 || Ex/s: 416.92

---------------------

Loading best model...
Training done.
===>  PREDICT Epoch 5


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:04:28


Finished Epoch 5 || Run Time:  112.7 | Load Time:  156.0 || F1:  96.97 | Prec:  95.20 | Rec:  98.80 || Ex/s: 678.48

Iteration:5 with training samples 240
Iteration:5 with prediction samples 182289
===>  PREDICT Epoch 5


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:01:16


Finished Epoch 5 || Run Time:   28.4 | Load Time:   48.6 || F1:  96.89 | Prec:  94.99 | Rec:  98.87 || Ex/s: 476.23



In [107]:
pred_transAct_ds['match_prediction'] = pred_transAct_ds['match_score'].apply(lambda score: 1 if score >= 0.5 else 0)
path_pred_transAct_rand_ds = os.path.join('.', 'Dissertation_Data_2', 'pred_transAct_ds.csv')
pred_transAct_ds.to_csv(path_pred_transAct_rand_ds)

In [105]:
pred_transAct_cor['match_prediction'] = pred_transAct_cor['match_score'].apply(lambda score: 1 if score >= 0.5 else 0)
path_pred_transAct_rand_cor = os.path.join('.', 'Dissertation_Data_2', 'pred_transAct_cor.csv')
pred_transAct_cor.to_csv(path_pred_transAct_rand_cor)

# Results

In [53]:
path_act_DS = os.path.join('.', 'Dissertation_Data_2', 'act_DS.csv')
mat_act_DS.to_csv(path_act_DS)

In [54]:
path_act_DS_rand = os.path.join('.', 'Dissertation_Data_2', 'act_DS_rand.csv')
mat_act_DS_rand.to_csv(path_act_DS_rand)

In [55]:
path_act_cora = os.path.join('.', 'Dissertation_Data_2', 'act_cora.csv')
mat_act_cora.to_csv(path_act_cora)

In [56]:
path_act_cora_rand = os.path.join('.', 'Dissertation_Data_2', 'act_cora_rand.csv')
mat_act_cora_rand.to_csv(path_act_cora_rand)

In [108]:
path_trans_DS = os.path.join('.', 'Dissertation_Data_2', 'res_trans_DS.csv')
mat_trans_DS.to_csv(path_trans_DS)

In [106]:
path_trans_cora = os.path.join('.', 'Dissertation_Data_2', 'res_trans_Cora.csv')
Mat_trans_cora.to_csv(path_trans_cora)